In [5]:
import requests
import time
import json

from inspirehep import *

In [6]:
f=open('db_LA.json','r')
db=json.load(f)
f.close()
idsLA=[d.get('LA_institution_id') for d in db]
idsAU=[d.get('author_id') for d in db]
db_index=get_index(db)

In [7]:
db_inst_index=get_index(db,index='LA_institution_id')

In [8]:
collaborations=[]
collaborations_old=[]

Pagination

In [9]:
page=0
if len(collaborations)>0 and  collaborations_old!=collaborations:
    cn_search='https://inspirehep.net/api/literature?sort=mostrecent&size=25&page=1&q=ac>100+and+de>2018'
    cn=f"{cn_search}+and+not+cn+{'+and+not+cn+'.join(collaborations)}"
else:
    #increase pagination
    if page==0:
        extra=''
    else:
        extra='+and+not+cn+'
    page=page+1
    cn_search=f'https://inspirehep.net/api/literature?sort=mostrecent&size=25&page={page}&q=ac>100+and+de>2018'
    cn=f"{cn_search}{extra}{'+and+not+cn+'.join(collaborations)}"

In [10]:
r=requests.get(cn)

In [11]:
d=r.json()

In [12]:
d.get('hits').get('total')

2520

In [13]:
l=d.get('hits').get('hits')

Not affilation in `'authors'` list for collaborations!

In [14]:
l=l[1:]

In [15]:
for ld in l:
    lit=ld.get('metadata')
    if not lit.get('collaborations'):
        continue # next paper
    try:
        collaboration=lit.get('collaborations')[0].get('value')
        FoS=[collaboration]
    except:
        collaboration=''
        FoS=[]

    if collaboration in collaborations:
        continue #next paper
        
    try:
        PAC=lit['primary_arxiv_category']
    except:
        PAC=[]
    la=lit.get('authors')
    for a in la:   
        CONTINUE,author_id,url_author,FoS,PAC=get_work(a,idsLA,idsAU,db_index,FoS,PAC)
        if CONTINUE:
            continue #next author
        positions,email_addresses,name,aids=get_author(url_author) #USES API!
        try:
            inst_id=[d.get('record').get('$ref').split('/')[-1] 
                 for d in positions if d.get('record') and d.get('current')][0]
        except:
            inst_id=None
        if inst_id not in idsLA:
            continue #next author
        di=db[db_inst_index[inst_id]]
        
        new={'author_id':author_id,
                 'name':name,
                 'LA_institution_id':inst_id,
                 'external_system_identifiers':di.get('external_system_identifiers'),
                 'LA_country':di.get('LA_country'),
                 'LA_institution':di.get('LA_institution'),
                 'email_addresses':email_addresses,
                 'positions':positions,
                 'ids':aids,
                 'primary_arxiv_category':PAC,
                 'Fields of Study': FoS # in ['Particle physics','Cosmology','Astrophysics','Mathematical physics']
                }
        db.append(new)
        break
    collaborations.append(collaboration) #Only after search for all the authors        
    break

NameError: name 'time' is not defined